In [100]:
import boto3
import json
from jinja2 import Environment, FileSystemLoader
import sys
from pathlib import Path
import os

In [101]:
template_dir = os.path.abspath(os.path.join(os.getcwd(),"../..","prompts-templates"))
template_dir

'/root/graph-visualizer/prompts-templates'

In [102]:
output_dir = os.path.abspath(os.path.join(os.getcwd(),"../..","model-results"))
output_dir

'/root/graph-visualizer/model-results'

In [103]:
template_filename = "story-prompt-with-ontology-v3.txt"

In [104]:
bedrock_runtime= boto3.client("bedrock-runtime",region_name="us-east-1")

In [105]:
environment = Environment(loader=FileSystemLoader(template_dir))
template = environment.get_template(template_filename)

In [106]:
prompt = template.render()

In [107]:
body=json.dumps(
{
    'prompt':f"\n\nHuman:{prompt}\n\nAssistant:",
    'top_p':0.5,
    'temperature':0.5,
    'top_k':5,
    'max_tokens_to_sample':3000
})

In [ ]:
response = bedrock_runtime.invoke_model(
    modelId='anthropic.claude-v2:1',
    contentType='application/json',
    accept='application/json',
    body=body
)

In [ ]:
response_body = json.loads(response['body'].read())

In [ ]:
response_body

{'completion': ' Here is the network graph JSON for the given context:\n\n```json\n{\n  "nodes": [\n    {\n      "id": "Monkey",\n      "attributes": "livesOn:BerryTree", \n      "type": "Animal",\n      "group": 2\n    },\n    {\n      "id": "BerryTree",\n      "attributes": "locatedAt:RiverBank",\n      "type": "Plant",\n      "group": 3  \n    },\n    {\n      "id": "RiverBank",\n      "attributes": "",\n      "type": "Location",\n      "group": 3\n    },\n    {  \n      "id": "Crocodile",\n      "attributes": "looked:hungry,looked:tired",\n      "type": "Animal", \n      "group": 2\n    },\n    {\n      "id": "Wife",\n      "attributes": "is:wicked,is:cunning",\n      "type": "Animal",\n      "group": 2\n    }, \n    {\n      "id": "Berries",  \n      "type": "Fruit",\n      "group": 4\n    },\n    {\n      "id": "Friendship",\n      "type": "Concept",\n      "group": 5\n    },\n    {\n      "id": "Trust",\n      "type": "Concept",\n      "group": 5\n    },\n    {\n      "id": "Bet

In [ ]:
same_name_count = int(len([f for f in os.listdir(output_dir) if f"claude-2-1-{template_filename.split('.txt')[0]}" in f])/2)

In [ ]:
file_count = "" if same_name_count ==0 else f'-{same_name_count}'

In [ ]:
with open(os.path.join(output_dir,f"claude-2-1-{template_filename.split('.txt')[0]}{file_count}.json"),"w+") as f:
    json.dump(response_body,f)

In [ ]:
with open(os.path.join(output_dir,f"claude-2-1-{template_filename.split('.txt')[0]}-graph{file_count}.json"),"w+") as f:
    json.dump(json.loads(response_body['completion'].split("```")[1].split("json")[1]),f)